In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

In [33]:
#reading the csv file
airbnb_data = pd.read_csv('AB_NYC_2019.csv')
#putting the csv file data in a df
airbnb_df = pd.DataFrame(airbnb_data)
airbnb_df.head()
#Creating a column for the minimum stay price per night
airbnb_df['min_price_to_stay'] = airbnb_df['price'] * airbnb_df['minimum_nights']
#cleaning data
airbnb_df = airbnb_df[['id', 'neighbourhood_group','neighbourhood',
 'room_type',
 'price',
 'minimum_nights',
 'min_price_to_stay',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'latitude',
 'longitude']]
#Getting only City Data
bronx_airbnb_df = airbnb_df.loc[airbnb_df['neighbourhood_group'] == 'Bronx']
bronx_airbnb_df.head()

,id,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,min_price_to_stay,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,latitude,longitude
171,44096,Bronx,Highbridge,Private room,40,1,40,219,2.04,3,353,40.83232,-73.93184
207,54626,Bronx,Highbridge,Private room,45,1,45,138,1.45,3,323,40.83075,-73.93058
260,63588,Bronx,Clason Point,Private room,90,2,180,0,NaN,7,349,40.81309,-73.85514
261,63610,Bronx,Eastchester,Entire home/apt,105,2,210,38,0.50,13,365,40.88057,-73.83572
309,71812,Bronx,Kingsbridge,Entire home/apt,90,30,2700,4,0.35,2,346,40.87207,-73.90193


In [34]:
# Number of records in the data
bronx_airbnb_df.count()

id                                1091
neighbourhood_group               1091
neighbourhood                     1091
room_type                         1091
price                             1091
minimum_nights                    1091
min_price_to_stay                 1091
number_of_reviews                 1091
reviews_per_month                  876
calculated_host_listings_count    1091
availability_365                  1091
latitude                          1091
longitude                         1091
dtype: int64

In [28]:
#Filling in all NA values with 0 reviews
bronx_airbnb_df.loc[:, 'reviews_per_month'] = bronx_airbnb_df.loc[:, 'reviews_per_month'].fillna(0)
bronx_airbnb_df.head()

/Users/amirabdallah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,min_price_to_stay,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
171,44096,Room with a View,190409,Waundell,Bronx,Highbridge,40.83232,-73.93184,Private room,40,1,40,219,2019-07-04,2.04,3,353
207,54626,Cozy bedroom by Yankee Stadium,190409,Waundell,Bronx,Highbridge,40.83075,-73.93058,Private room,45,1,45,138,2019-06-30,1.45,3,323
260,63588,LL3,295128,Carol Gloria,Bronx,Clason Point,40.81309,-73.85514,Private room,90,2,180,0,NaN,0.00,7,349
261,63610,DOMINIQUE'S NY mini efficiency* wifi*metro*quiet,310670,Vie,Bronx,Eastchester,40.88057,-73.83572,Entire home/apt,105,2,210,38,2019-06-27,0.50,13,365
309,71812,Condo Apartment with laundry in unit,369015,Thai,Bronx,Kingsbridge,40.87207,-73.90193,Entire home/apt,90,30,2700,4,2019-01-02,0.35,2,346


In [50]:
#Creating Bins for prices
price_bins = [0,49.9,99.9,149.9,180.9]
price_bin_names = ['0-49','50-99','100-149','150-180'] #used quartiles to cut out anything higher than 180
#placed prices in bins
bronx_airbnb_df['price range'] = pd.cut(bronx_airbnb_df['price'],price_bins,labels=price_bin_names)

min_nights_bronx_quartiles = bronx_airbnb_df['minimum_nights'].quantile([.25,.5,.75])
min_nights_lowerq = min_nights_bronx_quartiles[.25]
min_nights_upperq = min_nights_bronx_quartiles[.75]
min_nights_iqr = (min_nights_upperq-min_nights_lowerq).round(2)
min_nights_low_b = (min_nights_lowerq - (1.5*min_nights_iqr)).round(2)
print(f'lower bound is {min_nights_low_b}')
min_nights_up_b = (min_nights_upperq + (1.5*min_nights_iqr)).round(2)
print(f'upper bound is {min_nights_up_b}')

price_bronx_quartiles = bronx_airbnb_df['price'].quantile([.25,.5,.75])
price_lowerq = price_bronx_quartiles[.25]
price_upperq = price_bronx_quartiles[.75]
price_iqr = (price_upperq-price_lowerq).round(2)
price_low_b = (price_lowerq - (1.5*price_iqr)).round(2)
print(f'lower bound is {price_low_b}')
price_up_b = (price_upperq + (1.5*price_iqr)).round(2)
print(f'upper bound is {price_up_b}')




lower bound is -2.0
upper bound is 6.0
lower bound is -36.0
upper bound is 180.0


<ipython-input-50-c9f4c5abcef8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronx_airbnb_df['price range'] = pd.cut(bronx_airbnb_df['price'],price_bins,labels=price_bin_names)


In [55]:
over_180 = bronx_airbnb_df.loc[(bronx_airbnb_df['price']<=180) & (bronx_airbnb_df['minimum_nights']<=6)]
popularity_df=over_180.groupby(['neighbourhood','price range']).agg(['count']).reset_index()
#popularity_df = popularity_df.loc[(popularity_df['id']= )]
popularity_df

,neighbourhood,price range,id,neighbourhood_group,room_type,price,minimum_nights,min_price_to_stay,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,latitude,longitude
,,,count,count,count,count,count,count,count,count,count,count,count,count
0,Allerton,0-49,15,15,15,15,15,15,15,12,15,15,15,15
1,Allerton,50-99,12,12,12,12,12,12,12,12,12,12,12,12
2,Allerton,100-149,11,11,11,11,11,11,11,9,11,11,11,11
3,Allerton,150-180,0,0,0,0,0,0,0,0,0,0,0,0
4,Baychester,0-49,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Williamsbridge,150-180,0,0,0,0,0,0,0,0,0,0,0,0
188,Woodlawn,0-49,3,3,3,3,3,3,3,3,3,3,3,3
189,Woodlawn,50-99,8,8,8,8,8,8,8,8,8,8,8,8


In [44]:
bronx_airbnb_df.loc[:,'price range'] = pd.cut(bronx_airbnb_df['price'],price_bins,labels=price_bin_names)

/Users/amirabdallah/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
bronx_neighbourhood_by_popularity = []

for row in neighbourhood_by_count.head(25).index:
    if row[0] not in manhattan_neighbourhood_by_popularity:
        manhattan_neighbourhood_by_popularity.append(row[0])

manhattan_neighbourhood_listing_count = []


for n in manhattan_neighbourhood_by_popularity:
    count = len(manhattan_airbnb_df_1000.loc[manhattan_airbnb_df_1000['neighbourhood'] == n])
    manhattan_neighbourhood_listing_count.append(count)

listing_count_by_neighbourhood_df = pd.DataFrame({
    'neighbourhood': manhattan_neighbourhood_by_popularity,
    'count': manhattan_neighbourhood_listing_count
})

listing_count_by_neighbourhood_df = listing_count_by_neighbourhood_df.sort_values(by='count', ascending=False)
listing_count_by_neighbourhood_df

In [ ]:
manhattan_neighbourhood_by_popularity = []

for row in neighbourhood_by_count.head(25).index:
    if row[0] not in manhattan_neighbourhood_by_popularity:
        manhattan_neighbourhood_by_popularity.append(row[0])

manhattan_neighbourhood_listing_count = []


for n in manhattan_neighbourhood_by_popularity:
    count = len(manhattan_airbnb_df_1000.loc[manhattan_airbnb_df_1000['neighbourhood'] == n])
    manhattan_neighbourhood_listing_count.append(count)

listing_count_by_neighbourhood_df = pd.DataFrame({
    'neighbourhood': manhattan_neighbourhood_by_popularity,
    'count': manhattan_neighbourhood_listing_count
})

listing_count_by_neighbourhood_df = listing_count_by_neighbourhood_df.sort_values(by='count', ascending=False)
listing_count_by_neighbourhood_df

In [7]:
price_bins = [0,49.9,99.9,149.9,249.9,499.9,10001]
price_bin_names = ['0-49','50-99','100-149','150-249','250-499','>500']
airbnb_df['price range'] = pd.cut(airbnb_df['price'],price_bins,labels=price_bin_names)
neighbourhood_by_price = airbnb_df[['neighbourhood_group','price range','number_of_reviews','minimum_nights','availability_365']].groupby(['neighbourhood_group','price range']).mean()
neighbourhood_by_price

number_of_reviews  minimum_nights  \
neighbourhood_group price range                                      
Bronx               0-49                 25.153355        4.690096   
                    50-99                28.552063        5.084479   
                    100-149              26.736527        3.910180   
                    150-249              15.048387        2.774194   
                    250-499              16.333333        2.366667   
                    >500                  2.444444        2.444444   
Brooklyn            0-49                 16.468248        7.787226   
                    50-99                23.923360        5.472348   
                    100-149              29.251578        6.199126   
                    150-249              27.201180        5.760326   
                    250-499              20.082797        6.130225   
                    >500                 13.562724        7.405018   
Manhattan           0-49                 18.070735        9.830791   
                    50-99                26.642830        7.272488   
                    100-149              23.790432        8.578747   
                    150-249              19.366667        9.204147   
                    250-499              14.928858        9.049914   
                    >500                 10.447514        8.953591   
Queens              0-49                 27.610199        8.119274   
                    50-99                31.399926        3.918390   
                    100-149              24.551148        5.411273   
                    150-249              20.323729        4.303390   
                    250-499              19.391705        3.230415   
                    >500                  7.083333       25.805556   
Staten Island       0-49                 34.964706        4.258824   
                    50-99                34.798780        4.182927   
                    100-149              25.636364        7.590909   
                    150-249              25.473684        3.763158   
                    250-499              14.357143        5.428571   
                    >500                  0.166667        5.666667   

                                 availability_365  
neighbourhood_group price range                    
Bronx               0-49               151.897764  
                    50-99              168.787819  
                    100-149            169.520958  
                    150-249            154.129032  
                    250-499            255.366667  
                    >500               192.444444  
Brooklyn            0-49                98.220803  
                    50-99               94.243286  
                    100-149             96.766634  
                    150-249            105.750492  
                    250-499            122.974277  
                    >500               173.322581  
Manhattan           0-49                82.239945  
                    50-99               90.241250  
                    100-149             96.417866  
                    150-249            119.013978  
                    250-499            141.282559  
                    >500               181.192265  
Queens              0-49               144.797753  
                    50-99              140.923191  
                    100-149            136.496868  
                    150-249            147.577966  
                    250-499            201.391705  
                    >500               216.000000  
Staten Island       0-49               215.364706  
                    50-99              193.250000  
                    100-149            215.878788  
                    150-249            174.815789  
                    250-499            154.214286  
                    >500               238.500000

In [9]:
bronx_df = airbnb_df.loc[airbnb_df['neighbourhood_group'] == 'Bronx']

bronx_df_room_type = bronx_df[['room_type', 'neighbourhood','price', 'minimum_nights', 'number_of_reviews']].groupby(['room_type','neighbourhood',]).agg(['mean','count', 'min', 'max'])
sorted_bronx_df = bronx_df_room_type.sort_values(by=('price', 'mean'), ascending=False)

# bronx_df.loc[bronx_df['neighbourhood'] == 'Allerton'][['room_type', 'neighbourhood','price', 'minimum_nights']].groupby(['room_type','neighbourhood',]).agg(['mean','count', 'min', 'max'])
sorted_bronx_df = sorted_bronx_df.loc[(sorted_bronx_df[('price', 'mean')] <= 400) & (sorted_bronx_df[('price', 'mean')] >= 100)]

sorted_bronx_df

price                   \
                                          mean count  min   max   
room_type       neighbourhood                                     
Entire home/apt City Island         235.272727    11   84  1000   
                Eastchester         233.166667     6  105   475   
                Belmont             228.666667     3  140   299   
                Westchester Square  221.750000     4   70   670   
                Van Nest            216.400000     5  100   425   
                Spuyten Duyvil      186.333333     3   79   360   
                Unionport           183.750000     4   85   450   
                Clason Point        171.750000     8   75   379   
                West Farms          165.000000     1  165   165   
                Hunts Point         150.000000     1  150   150   
                Mount Eden          150.000000     1  150   150   
                Williamsbridge      148.315789    19   63   500   
                Riverdale           141.500000     6   85   250   
                Wakefield           139.388889    18   28   309   
                University Heights  131.000000     5  100   175   
                Claremont Village   130.500000     8   57   390   
                Port Morris         129.062500    16   90   220   
                Morrisania          129.000000     5   50   225   
                Kingsbridge         128.611111    18   47   399   
                East Morrisania     126.666667     3   60   200   
                North Riverdale     126.666667     3  105   150   
                Allerton            123.062500    16   49   450   
                Fordham             121.615385    13   65   223   
                Throgs Neck         121.545455    11   74   325   
                Norwood             118.833333     6   58   200   
                Concourse           116.650000    20   60   250   
                Pelham Bay          115.916667    12   37   299   
                Longwood            112.850000    20   40   350   
                Pelham Gardens      112.294118    17   40   450   
                Mott Haven          110.000000    27   65   150   
                Concourse Village   109.555556     9   65   159   
                Mount Hope          107.900000    10   48   250   
                Highbridge          107.571429     7   70   240   
                Morris Heights      105.250000     4   42   150   
                Morris Park         103.500000     4   39   190   
Private room    Olinville           102.500000     2   80   125   
Entire home/apt Parkchester         100.444444     9   65   195   
Shared room     Pelham Bay          100.000000     2   50   150   

                                   minimum_nights                 \
                                             mean count min  max   
room_type       neighbourhood                                      
Entire home/apt City Island              1.909091    11   1    3   
                Eastchester              1.666667     6   1    2   
                Belmont                  1.666667     3   1    2   
                Westchester Square       1.500000     4   1    2   
                Van Nest                 2.000000     5   1    5   
                Spuyten Duyvil          63.333333     3   2  186   
                Unionport                2.000000     4   2    2   
                Clason Point             2.000000     8   1    3   
                West Farms               2.000000     1   2    2   
                Hunts Point              2.000000     1   2    2   
                Mount Eden               1.000000     1   1    1   
                Williamsbridge           5.052632    19   1   30   
                Riverdale                7.666667     6   1   30   
                Wakefield                6.111111    18   1   30   
                University Heights       3.200000     5   1    7   
                Claremont Village       46.250000     8   2   90   
 

In [12]:
sorted_bronx_df.mean()

price              mean     141.112892
                   count      8.868421
                   min       74.605263
                   max      302.552632
minimum_nights     mean       7.009494
                   count      8.868421
                   min        1.157895
                   max       24.368421
number_of_reviews  mean      24.593938
                   count      8.868421
                   min        1.526316
                   max       92.605263
dtype: float64